In [ ]:
from glob import glob
import re

from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
gtes = sorted(glob('../data/OTB100GT/*'))
files = sorted(glob('../data/OTB100/*'))

In [ ]:
assert len(gtes) == len(files)

In [ ]:
def loadgt(file):
    with open(file) as f:
        return np.asarray([list(map(int, re.findall(r'[\w]+', line.strip()))) for line in f.readlines()])

In [ ]:
gtboxes = [loadgt(file + '/groundtruth_rect.txt') for file in gtes]

In [ ]:
results = [loadmat(file)['results'][0][0][0][0][0] for file in files]

In [ ]:
def location(rect1, rect2):
    cx1, cy1 = rect1[0] + rect1[2]/2, rect1[1] + rect1[3]/2
    cx2, cy2 = rect2[0] + rect2[2]/2, rect2[1] + rect2[3]/2
    return ((cy2-cy1)**2 + (cx2-cx1)**2)**.5

In [ ]:
def overlap(rect1, rect2):
    left = np.maximum(rect1[0], rect2[0])
    right = np.minimum(rect1[0] + rect1[2], rect2[0] + rect2[2])
    top = np.maximum(rect1[1], rect2[1])
    bottom = np.minimum(rect1[1] + rect1[3], rect2[1] + rect2[3])

    intersect = np.maximum(0, right - left) * np.maximum(0, bottom - top)
    union = rect1[2] * rect1[3] + rect2[2] * rect2[3] - intersect
    iou = np.clip(intersect / union, 0, 1)
    return iou

In [ ]:
iou = np.asarray([overlap(g, b) for gt, bb in zip(gtboxes, results) for g, b in zip(gt, bb)])

In [ ]:
x_iou = np.arange(0, 1.001, .001)

In [ ]:
perf_iou = [iou[np.where(iou>=flag)].size / iou.size for flag in x_iou]

In [ ]:
plt.plot(x_iou, perf_iou, label='MDNet [%.3f]' % (sum(perf_iou) / len(perf_iou)))
plt.legend()
plt.show()

In [ ]:
cle = np.asarray([location(g, b) for gt, bb in zip(gtboxes, results) for g, b in zip(gt, bb)])

In [ ]:
x_cle = np.arange(0, 50, .1)

In [ ]:
perf_cle = [cle[np.where(cle<=flag)].size / cle.size for flag in x_cle]

In [ ]:
plt.plot(x_cle, perf_cle, label='MDNet [%.3f]' % (sum(perf_cle) / len(perf_cle)))
plt.legend()
plt.show()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.plot(x_cle, perf_cle, label='%.3f' % (sum(perf_cle) / len(perf_cle)))
ax1.set_title('CLE')
ax1.legend()
ax2.plot(x_iou, perf_iou, label='%.3f' % (sum(perf_iou) / len(perf_iou)))
ax2.set_title('IoU')
ax2.legend()

So, we get performance in pyMDNet implementation as below.

|           | CLE  | IOU  |
| --------- | ---- | ---- |
| paper     | .909 | .678 |
| implement | .893 | .763 |

In [31]:
import sys
sys.path.append("/home/maybe/pymot")
from pymot import MOTEvaluation
import json
from collections import defaultdict
from glob import glob
from os import path
import pickle

In [32]:
def mot(groundtruth, hypotheses):
    evaluator = MOTEvaluation(groundtruth, hypotheses, 0.2)
    evaluator.evaluate()
    print(evaluator.getMOTA())
    print(evaluator.getMOTP()
    evaluator.getRelativeStatistics()
    evaluator.getAbsoluteStatistics()

In [36]:
def load_result():
    data_path = '/home/maybe/MOT17/train/MOT17-02-DPM/'
    seqs = defaultdict(dict)
    objs = defaultdict(dict)
    with open(path.join(data_path, 'gt', 'gt.txt')) as f:
        for seq, obj, *data in [line.strip().split(',') for line in f.readlines()]:
            if int(obj) < 16:
                seqs[int(seq)][int(obj)] = data
                objs[int(obj)][int(seq)] = data
    pb = defaultdict(dict)
    for file in glob('../results_*.p'):
        obj_id = int(file.split('_')[1].split('.')[0])
        with open(file, 'rb') as f:
            r = pickle.load(f)
        pb[obj_id] = r
    rb = defaultdict(dict)
    for obj, values in objs.items():
        for v, d in values.items():
            if v < len(pb[obj]):
                rb[v][obj] = pb[obj][v]
    return rb, seqs

In [37]:
def result_to_json(rb, seqs):
    hypotheses = {}
    groundtruth = {}
    num = 0
    hypotheses['frames'] = []
    hypotheses['class'] = "video"
    hypotheses['filename'] = "/home/maybe/MOT/labs/mot17.idx"
    groundtruth['frames'] = []
    groundtruth['class'] = "video"
    groundtruth['filename'] = "/home/maybe/MOT/labs/mot17.idx"
    for seq, oo in rb.items():
        if not seq: continue
        # hypotheses
        hypotheses["frames"].append({"timestamp": num})
        hypotheses["frames"][num]["num"] = num
        hypotheses["frames"][num]["class"] = "frame"
        hypotheses["frames"][num]["hypotheses"] = []
        idx = 0
        for o, value in seqs[seq].items():
            x, y, w, h, *_ = map(float, value)
            hypotheses["frames"][num]["hypotheses"].append({"height": h, "width": w, "id": idx, "y": y, "x":x})
            idx += 1
        # groundtruth
        groundtruth["frames"].append({"timestamp": num})
        groundtruth["frames"][num]["num"] = num
        groundtruth["frames"][num]["class"] = "frame"
        groundtruth["frames"][num]["annotations"] = []
        idx = 0
        for o, value in oo.items():
            x, y, w, h, *_ = map(float, value)
            groundtruth["frames"][num]["annotations"].append({"dco": False, "height": h, "width": w, "id": idx, "y": y, "x":x})
            idx += 1
        num += 1
    return groundtruth, hypotheses

In [38]:
rb, seqs = load_result()
groundtruth, hypotheses = result_to_json(rb, seqs)
mot(groundtruth, hypotheses)